In [20]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import time
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

import sklearn.metrics

from helper import Helper
import utils

In [2]:
# Read in data
X_train, maxLength, y_train = Helper(0).getPadding3D()

Reading from: ../data/training_set_rel3.tsv 
Word vector shape: (17033, 100)
Number of instances:  1783
Max essay length: 789
(0, 789, 100)
(1, 789, 100)
(2, 789, 100)
(3, 789, 100)
(4, 789, 100)
(5, 789, 100)
(6, 789, 100)
(7, 789, 100)
(8, 789, 100)
(9, 789, 100)
(10, 789, 100)
(11, 789, 100)
(12, 789, 100)
(13, 789, 100)
(14, 789, 100)
(15, 789, 100)
(16, 789, 100)
(17, 789, 100)
(18, 789, 100)
(19, 789, 100)
(20, 789, 100)
(21, 789, 100)
(22, 789, 100)
(23, 789, 100)
(24, 789, 100)
(25, 789, 100)
(26, 789, 100)
(27, 789, 100)
(28, 789, 100)
(29, 789, 100)
(30, 789, 100)
(31, 789, 100)
(32, 789, 100)
(33, 789, 100)
(34, 789, 100)
(35, 789, 100)
(36, 789, 100)
(37, 789, 100)
(38, 789, 100)
(39, 789, 100)
(40, 789, 100)
(41, 789, 100)
(42, 789, 100)
(43, 789, 100)
(44, 789, 100)
(45, 789, 100)
(46, 789, 100)
(47, 789, 100)
(48, 789, 100)
(49, 789, 100)
(50, 789, 100)
(51, 789, 100)
(52, 789, 100)
(53, 789, 100)
(54, 789, 100)
(55, 789, 100)
(56, 789, 100)
(57, 789, 100)
(58, 789, 100)

(512, 789, 100)
(513, 789, 100)
(514, 789, 100)
(515, 789, 100)
(516, 789, 100)
(517, 789, 100)
(518, 789, 100)
(519, 789, 100)
(520, 789, 100)
(521, 789, 100)
(522, 789, 100)
(523, 789, 100)
(524, 789, 100)
(525, 789, 100)
(526, 789, 100)
(527, 789, 100)
(528, 789, 100)
(529, 789, 100)
(530, 789, 100)
(531, 789, 100)
(532, 789, 100)
(533, 789, 100)
(534, 789, 100)
(535, 789, 100)
(536, 789, 100)
(537, 789, 100)
(538, 789, 100)
(539, 789, 100)
(540, 789, 100)
(541, 789, 100)
(542, 789, 100)
(543, 789, 100)
(544, 789, 100)
(545, 789, 100)
(546, 789, 100)
(547, 789, 100)
(548, 789, 100)
(549, 789, 100)
(550, 789, 100)
(551, 789, 100)
(552, 789, 100)
(553, 789, 100)
(554, 789, 100)
(555, 789, 100)
(556, 789, 100)
(557, 789, 100)
(558, 789, 100)
(559, 789, 100)
(560, 789, 100)
(561, 789, 100)
(562, 789, 100)
(563, 789, 100)
(564, 789, 100)
(565, 789, 100)
(566, 789, 100)
(567, 789, 100)
(568, 789, 100)
(569, 789, 100)
(570, 789, 100)
(571, 789, 100)
(572, 789, 100)
(573, 789, 100)
(574, 78

(1023, 789, 100)
(1024, 789, 100)


In [3]:
N_CLASSES = 12
N_HIDDEN = 128

# Define paramaters for the model
LEARNING_RATE = 0.001
BATCH_SIZE = 128
SKIP_STEP = 1
DROPOUT = 0.75
N_EPOCHS = 5

In [4]:
def one_hot(Y):
    output = np.zeros((Y.shape[0], N_CLASSES), dtype=int)
    for i in range(Y.shape[0]):
        output[i][int(Y[i])-1]=1
    
    return output
        
Y_train = one_hot(y_train)
print(Y_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
def one_hot_reverse(Y):
    output = np.argwhere(Y==1)[:,1]+1
    
    return output
    

In [5]:
with tf.name_scope('data'):
    X = tf.placeholder(tf.float32, [None, maxLength, 100], name="X_placeholder")
    Y = tf.placeholder(tf.float32, [None, N_CLASSES], name="Y_placeholder")

#state = tf.placeholder(tf.float32, shape=[None_preds, 2*N_HIDDEN])
dropout_keep_prob = tf.placeholder(tf.float32, name='dropout_keep_prob')

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')

In [6]:
with tf.name_scope('process_data'):
    w1 = tf.Variable(tf.truncated_normal(shape=[100, 128], stddev=1.0))
    b1 = tf.Variable(tf.constant(0.1, shape=[128]))

    _X = tf.transpose(X, perm=[1, 0, 2]) # X: BATCH_SIZE*maxLength*100
    _X = tf.reshape(_X, shape=[-1, 100])

    _X = tf.nn.xw_plus_b(_X, w1, b1)

    _X = tf.split(_X, maxLength, 0)

In [7]:
with tf.name_scope('lstm') as scope:
    lstm_cell = tf.contrib.rnn.LSTMCell(N_HIDDEN, forget_bias=1.0, state_is_tuple=False)
    cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob=dropout_keep_prob)
    outputs, _ = tf.nn.static_rnn(cell=cell, inputs=_X, dtype=tf.float32)


with tf.variable_scope('softmax_linear') as scope:
    w2 = tf.Variable(tf.truncated_normal(shape=[N_HIDDEN, N_CLASSES]))
    b2 = tf.Variable(tf.constant(0.1, shape=[N_CLASSES]))

    logits = tf.nn.xw_plus_b(outputs[-1], w2, b2, name='logits')


with tf.name_scope('loss') as scope:
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name='loss')


with tf.name_scope('optimizer') as scope:
    #: define training op
    optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE).minimize(loss, global_step=global_step)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [23]:
with tf.name_scope('accuracy') as scope:
    score_class = np.ones((BATCH_SIZE,1)) * np.arange(1, 13)
    prob = tf.nn.softmax(logits)
    preds = tf.reduce_sum(tf.multiply(prob, score_class), 1)
    
    correct_preds = tf.equal(tf.to_int64(tf.round(preds)), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    #preds = tf.argmax(tf.nn.softmax(logits), 1)
    #correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
    #accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
    

In [28]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver = tf.train.Saver()
    # to visualize using TensorBoard
    #writer = tf.summary.FileWriter('./graphs/aes', sess.graph)
    ##### You have to create folders to store checkpoints
    #ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
    # if that checkpoint exists, restore from checkpoint
    #if ckpt and ckpt.model_checkpoint_path:
    #    saver.restore(sess, ckpt.model_checkpoint_path)

    initial_step = global_step.eval()

    start_time = time.time()
    
    total_loss = 0.0
    index = 0
    total_correct_preds = 0
    for i in range(N_EPOCHS):  # train the model n_epochs times
        machine_score = np.array([])
        human_score = np.array([])
        
        for X_batch, Y_batch in utils.get_batches(X_train, Y_train, BATCH_SIZE):
            X_batch = X_batch.reshape((BATCH_SIZE, maxLength, 100))
            Y_batch = Y_batch.reshape((BATCH_SIZE, N_CLASSES))
            
            _, loss_batch, _accuracy, _preds= sess.run([optimizer, loss, accuracy, preds],
                                     feed_dict={X: X_batch, Y: Y_batch, dropout_keep_prob: DROPOUT})
            
            total_loss += loss_batch
            
            machine_score = np.append(machine_score, _preds, axis=0)
            y_score = one_hot_reverse(Y_batch)
            print(y_score)
            human_score = np.append(human_score, y_score, axis=0)
            
            print('Predictions:',machine_score)
            print('Accuracy:',_accuracy)
            index +=1
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
                total_loss = 0.0
                #saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)
        
        print(machine_score)
        print(human_score)
        score = sklearn.metrics.cohen_kappa_score(np.round(machine_score), np.round(human_score), weights='quadratic')
        print("Kappa score: ", score)
        
    print("Optimization Finished!")  # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))

    # test the model
    '''
    n_batches = int(mnist.test.num_examples / BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits],
                                               feed_dict={X: X_batch, Y: Y_batch, dropout: DROPOUT})
        total_correct_preds += sess.run(accuracy)

    print("Accuracy {0}".format(total_correct_preds / mnist.test.num_examples))
    '''

[ 8  9  7 10  8  8 10 10  9  9  8  8  7  6  6 12  8  8  4  6  8  3 10 11
  8  9  4  9  9  8 10 10  6  8  9 10 12  8 10  7  2  8  6  8  8  8  8 11
  6  5  9  7  8 10  8 10  9  7  8  4  8  8  8  7  9  9  8  9  7 12 10 10
  8  7  8  8 10 10 10  8  8  8  7  6 10  8 10  9  6  7  8 11 11  8 10  7
  8 11  8  7 10  8  9 10  9 11 10  8  8  6 11  9  8  8  9  4  8 12 10  8
  8  9 10  7  5  8  9  9]
Predictions: [10.32405472  4.29616642  6.60229301  2.00208688  9.63981533  4.3655386
 10.9683218  10.65299702  5.1565938   7.59705544  6.08940983  7.35394239
  2.23787737  2.00078368  2.2226913   3.03281689 10.5036087   2.50978923
  6.60825586 10.10185051  3.54451466  6.28410625  7.95892096  3.76251268
  6.96361351  7.55400896  1.70367241  2.98069549  8.24380112  2.06033349
  2.9444747   2.56105947  8.89252186  6.74115419  6.44997025 11.87501717
  9.65580368  9.99992657  1.86910748  6.12829781  3.0997858  10.94459438
  4.22612095  2.0904882   1.78211439  4.07550669  5.03914165  2.77165461
 10.96960354 

[ 8  7  8 10 12  8  8  7  6  7 10 10  8  8  6  8  8  8  9  8  8  8  8  8
 12 10  9  8  7  6  8 12  6  8  6  7  9  9 10  9 11  8  9  8 11  8 11  7
 11  8  8  8 10  8  8 10  8  8  8 10 11  9  2  7  8  7  9  8  4  7 10  9
  8  8  8  8  8  7  8  9  8  8  9  8  8 12  7  6  9  8  6  8 10  8 10  9
  9  7 10  9  8  8  9  8  8  7  8  9  8  9  9 12  9 10  8  7  8 10 10  7
 10  9 11  7  8  8  8  8]
Predictions: [10.32405472  4.29616642  6.60229301  2.00208688  9.63981533  4.3655386
 10.9683218  10.65299702  5.1565938   7.59705544  6.08940983  7.35394239
  2.23787737  2.00078368  2.2226913   3.03281689 10.5036087   2.50978923
  6.60825586 10.10185051  3.54451466  6.28410625  7.95892096  3.76251268
  6.96361351  7.55400896  1.70367241  2.98069549  8.24380112  2.06033349
  2.9444747   2.56105947  8.89252186  6.74115419  6.44997025 11.87501717
  9.65580368  9.99992657  1.86910748  6.12829781  3.0997858  10.94459438
  4.22612095  2.0904882   1.78211439  4.07550669  5.03914165  2.77165461
 10.96960354 

[10  8  8  9 11  6  8 10  8  9  8 10  7  9  9  8 10 11  8  6  8  9  7  8
  9 10  8  8  8  9  6  8  8  8  8  8  8  9  8 11 10  8  8  9  8  9  8  9
  6  6  8  8  9  8  8  6 12  8  8  6  8  7  8 10  8  9 10 11 10  9 10  9
  8  9  8 10  7 10  9 10 10  8  6 10  6  8  7 10  8  9  8 10  9 10  8  6
  9  8  8 10 10  9  9  8  8  8 10  8  8  7  8  9  9  8  9  8 10  9  8 10
  8  9  7  8  8  8 10  8]
Predictions: [10.32405472  4.29616642  6.60229301  2.00208688  9.63981533  4.3655386
 10.9683218  10.65299702  5.1565938   7.59705544  6.08940983  7.35394239
  2.23787737  2.00078368  2.2226913   3.03281689 10.5036087   2.50978923
  6.60825586 10.10185051  3.54451466  6.28410625  7.95892096  3.76251268
  6.96361351  7.55400896  1.70367241  2.98069549  8.24380112  2.06033349
  2.9444747   2.56105947  8.89252186  6.74115419  6.44997025 11.87501717
  9.65580368  9.99992657  1.86910748  6.12829781  3.0997858  10.94459438
  4.22612095  2.0904882   1.78211439  4.07550669  5.03914165  2.77165461
 10.96960354 

[ 9 10  8  8  7  7 10  9  8  9 10 11  7 10  8  8  8  8  7  9  8  8 11  8
  8  8 10  9  8 10  9  8  6  8  9  6 11  7  9  8  8  8  8  8 10  9 12  8
  8  5  9  8  8  9  6 10  6  9  8 10  8  2  9 10 10  8  8  5 10  8  7  8
  8  8 10  8 12  6  7 10  9 10  9  8  8 10  8  9 12  8  8  8 10 10  8  7
  8 12  6  9 10  8  9 12 11  8 11  9  8 10  8  9  9  8  7 11  9 10  8  8
  9 10  4 10 10  8  9  8]
Predictions: [10.32405472  4.29616642  6.60229301  2.00208688  9.63981533  4.3655386
 10.9683218  10.65299702  5.1565938   7.59705544  6.08940983  7.35394239
  2.23787737  2.00078368  2.2226913   3.03281689 10.5036087   2.50978923
  6.60825586 10.10185051  3.54451466  6.28410625  7.95892096  3.76251268
  6.96361351  7.55400896  1.70367241  2.98069549  8.24380112  2.06033349
  2.9444747   2.56105947  8.89252186  6.74115419  6.44997025 11.87501717
  9.65580368  9.99992657  1.86910748  6.12829781  3.0997858  10.94459438
  4.22612095  2.0904882   1.78211439  4.07550669  5.03914165  2.77165461
 10.96960354 

[ 8 10  8 10  8  8  6 11  9 10 10  8  8 10 10  9 10 11  6 10 11  8  8  8
  6  8  7  7  7 10 10  8  8 12  8  8  9  7 11 11  6  8 12  9  9  8  8  8
 10  9  8 10  7  8 12 10  8  9  9  8  9  7  8  9  8 11  7  7 11  9  8 11
  9  8  6  2  8  8  7  8 10 10  8  8  8  8  9 11  8  8  9  8  6  5  9  9
  4  7 10  9 12 12  8  8  9  6  9  8 10 10  8  8  8  9  7  8 10  8  8 10
 10  8  9 11  8  9  8 10]
Predictions: [10.32405472  4.29616642  6.60229301 ...  8.01242638 11.19394398
  8.00331402]
Accuracy: 0.203125
Average loss at step 9:   4.5
[10.32405472  4.29616642  6.60229301 ...  8.01242638 11.19394398
  8.00331402]
[ 8.  9.  7. ...  9.  8. 10.]
Kappa score:  0.03722890222027686
[ 8  9  7 10  8  8 10 10  9  9  8  8  7  6  6 12  8  8  4  6  8  3 10 11
  8  9  4  9  9  8 10 10  6  8  9 10 12  8 10  7  2  8  6  8  8  8  8 11
  6  5  9  7  8 10  8 10  9  7  8  4  8  8  8  7  9  9  8  9  7 12 10 10
  8  7  8  8 10 10 10  8  8  8  7  6 10  8 10  9  6  7  8 11 11  8 10  7
  8 11  8  7 10  8  9 10  9 11 10

...

[ 8 10  8 10  8  8  6 11  9 10 10  8  8 10 10  9 10 11  6 10 11  8  8  8
  6  8  7  7  7 10 10  8  8 12  8  8  9  7 11 11  6  8 12  9  9  8  8  8
 10  9  8 10  7  8 12 10  8  9  9  8  9  7  8  9  8 11  7  7 11  9  8 11
  9  8  6  2  8  8  7  8 10 10  8  8  8  8  9 11  8  8  9  8  6  5  9  9
  4  7 10  9 12 12  8  8  9  6  9  8 10 10  8  8  8  9  7  8 10  8  8 10
 10  8  9 11  8  9  8 10]
Predictions: [ 8.56653595  7.95268297  8.03167725 ...  7.07064819  7.96020317
 10.65138435]
Accuracy: 0.2109375
Average loss at step 33:   1.9
[ 8.56653595  7.95268297  8.03167725 ...  7.07064819  7.96020317
 10.65138435]
[ 8.  9.  7. ...  9.  8. 10.]
Kappa score:  0.24984837877004862
[ 8  9  7 10  8  8 10 10  9  9  8  8  7  6  6 12  8  8  4  6  8  3 10 11
  8  9  4  9  9  8 10 10  6  8  9 10 12  8 10  7  2  8  6  8  8  8  8 11
  6  5  9  7  8 10  8 10  9  7  8  4  8  8  8  7  9  9  8  9  7 12 10 10
  8  7  8  8 10 10 10  8  8  8  7  6 10  8 10  9  6  7  8 11 11  8 10  7
  8 11  8  7 10  8  9 10  9 11 

[ 8  7  8 10 12  8  8  7  6  7 10 10  8  8  6  8  8  8  9  8  8  8  8  8
 12 10  9  8  7  6  8 12  6  8  6  7  9  9 10  9 11  8  9  8 11  8 11  7
 11  8  8  8 10  8  8 10  8  8  8 10 11  9  2  7  8  7  9  8  4  7 10  9
  8  8  8  8  8  7  8  9  8  8  9  8  8 12  7  6  9  8  6  8 10  8 10  9
  9  7 10  9  8  8  9  8  8  7  8  9  8  9  9 12  9 10  8  7  8 10 10  7
 10  9 11  7  8  8  8  8]
Predictions: [ 8.05424976  8.28808784  8.05044556  7.22936678  8.03654099  7.87011194
  9.72540665  9.47027874  8.25292492  9.17523766  9.20168877  7.31187725
  8.84065723  7.87987137  8.46882629 11.0245142   9.15283108  8.01668835
  6.672328    6.29605961  8.74668312  9.11145496  8.0846796   9.82806396
  8.74501801  8.99700069  4.4469223   8.2511692   7.74316788  9.7641201
  8.83336449  9.43046665  5.07047606  8.01186657  8.82134342  5.12973785
  9.09573174  8.7258091   9.77108002  7.41019392  7.97929907  8.5252285
  7.56804943  7.94272089  9.14906502  8.19221497  8.50475788  9.33974838
 10.43826866  

[10  8  8  9 11  6  8 10  8  9  8 10  7  9  9  8 10 11  8  6  8  9  7  8
  9 10  8  8  8  9  6  8  8  8  8  8  8  9  8 11 10  8  8  9  8  9  8  9
  6  6  8  8  9  8  8  6 12  8  8  6  8  7  8 10  8  9 10 11 10  9 10  9
  8  9  8 10  7 10  9 10 10  8  6 10  6  8  7 10  8  9  8 10  9 10  8  6
  9  8  8 10 10  9  9  8  8  8 10  8  8  7  8  9  9  8  9  8 10  9  8 10
  8  9  7  8  8  8 10  8]
Predictions: [ 8.05424976  8.28808784  8.05044556  7.22936678  8.03654099  7.87011194
  9.72540665  9.47027874  8.25292492  9.17523766  9.20168877  7.31187725
  8.84065723  7.87987137  8.46882629 11.0245142   9.15283108  8.01668835
  6.672328    6.29605961  8.74668312  9.11145496  8.0846796   9.82806396
  8.74501801  8.99700069  4.4469223   8.2511692   7.74316788  9.7641201
  8.83336449  9.43046665  5.07047606  8.01186657  8.82134342  5.12973785
  9.09573174  8.7258091   9.77108002  7.41019392  7.97929907  8.5252285
  7.56804943  7.94272089  9.14906502  8.19221497  8.50475788  9.33974838
 10.43826866  

[ 9 10  8  8  7  7 10  9  8  9 10 11  7 10  8  8  8  8  7  9  8  8 11  8
  8  8 10  9  8 10  9  8  6  8  9  6 11  7  9  8  8  8  8  8 10  9 12  8
  8  5  9  8  8  9  6 10  6  9  8 10  8  2  9 10 10  8  8  5 10  8  7  8
  8  8 10  8 12  6  7 10  9 10  9  8  8 10  8  9 12  8  8  8 10 10  8  7
  8 12  6  9 10  8  9 12 11  8 11  9  8 10  8  9  9  8  7 11  9 10  8  8
  9 10  4 10 10  8  9  8]
Predictions: [ 8.05424976  8.28808784  8.05044556  7.22936678  8.03654099  7.87011194
  9.72540665  9.47027874  8.25292492  9.17523766  9.20168877  7.31187725
  8.84065723  7.87987137  8.46882629 11.0245142   9.15283108  8.01668835
  6.672328    6.29605961  8.74668312  9.11145496  8.0846796   9.82806396
  8.74501801  8.99700069  4.4469223   8.2511692   7.74316788  9.7641201
  8.83336449  9.43046665  5.07047606  8.01186657  8.82134342  5.12973785
  9.09573174  8.7258091   9.77108002  7.41019392  7.97929907  8.5252285
  7.56804943  7.94272089  9.14906502  8.19221497  8.50475788  9.33974838
 10.43826866  

[ 8 10  8 10  8  8  6 11  9 10 10  8  8 10 10  9 10 11  6 10 11  8  8  8
  6  8  7  7  7 10 10  8  8 12  8  8  9  7 11 11  6  8 12  9  9  8  8  8
 10  9  8 10  7  8 12 10  8  9  9  8  9  7  8  9  8 11  7  7 11  9  8 11
  9  8  6  2  8  8  7  8 10 10  8  8  8  8  9 11  8  8  9  8  6  5  9  9
  4  7 10  9 12 12  8  8  9  6  9  8 10 10  8  8  8  9  7  8 10  8  8 10
 10  8  9 11  8  9  8 10]
Predictions: [8.05424976 8.28808784 8.05044556 ... 9.4759655  8.46969795 9.99802017]
Accuracy: 0.1484375
Average loss at step 41:   1.8
[8.05424976 8.28808784 8.05044556 ... 9.4759655  8.46969795 9.99802017]
[ 8.  9.  7. ...  9.  8. 10.]
Kappa score:  0.2754156403407575
Optimization Finished!
Total time: 181.38641214370728 seconds
